### Task 6: 
Create an interactive model of the real, inverted image of an object placed outside the focal range of an ideal thin lens

below is an amalgamation of matlab and python code - need to clean-up

In [ ]:
#converging_thin_lens
# Determines the image of an object in a thin converging lens.
    # (i.e. a bi-convex lens formed by two spherical sections)
# Assume spacing is negligible.

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os, decimal
from decimal import Decimal


def load_img(image_path):
    """loads image file and converts it to a numpy array."""
    try:
        global img
        img = Image.open(image_path) 
        return np.array(img)
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

def converging_thin_lens():
    global d
    h.lens = 1          #Height of lens
    Rc = 5              #Radius of curvature of lenses (for visuals only)
    w.lens = 0.1        #Width of lens (for visual only)
    f.lens = 0.6        #Focal length of lens
    x0, y0 = 1 , 0.0    #Object centre
    w = 0.5             #Width of object (this scales the image)
    d.delta = 0.02      #Movement on keypress
    fsize = 18          #Fontsize
    d.overlay_rays=1    #Overlay rays
    
    #Import bitmap image of object
    image_path = "stinkbug.png"  # replace this image file if needed
    if not os.path.exists(image_path): # error handling
        print(f"'{image_path}' not found")
        raise SystemExit
    else:
        orig_img = load_img(image_path)
        if orig_img is None:
            return
        print(f"{image_path} successfully loaded")
    
    orig_img = double(np.flipdim(orig_img,1))/255
    
    height, width = orig_img.shape[:2]
        '''dim = size(orig_img)
        width = dim(2)
        height = dim(1)'''
    aspect_ratio = height / width
    #Determine x,y coordinates of pixel locations
    d.x = np.linspace(0, w, width) - w/2 + x0
    d.y = np.linspace(0, w*aspect_ratio, height) - 0.5*w*aspect_ratio + y0 
    [d.x,d.y] = np.meshgrid(d.x, d.y)

    #Detrmine most left x coordinate of object
    x.min = x0 - w/2

    #Determine largest and smallest y coordinates of object
    y.max = 0.5*w*aspect_ratio + y0
    y.min = -0.5*w*aspect_ratio + y0

    #Create figure and plot object
    d.fig = figure( 'KeyPressFcn',@keypress )
    d.Obj = surf(d.x,d.y,-ones(height,width),orig_img)
    shading interp
    view(2)
    hold on
    axis equal
    xlabel('x','fontsize',fsize)
    ylabel('y','fontsize',fsize)
    title('Image of object in a converging lens','fontsize',fsize)
    set(gca,'fontsize',fsize)
    xlim([-5*f.lens,5*f.lens])
    ylim([-d.delta-2*h.lens,+d.delta+2*h.lens])
    axis manual
    box on

    #Determin x coordinate of centre of curvature for lenses
    xL = sqrt( Rc^2 - h.lens^2 ) - w.lens/2
    xR = - sqrt( Rc^2 - h.lens^2 ) + w.lens/2

    #Determine lens arcs
    tmax = atan( h.lens/(xL + w.lens/2) ) # arctan (height of length / (centre of curvature for left side of lens + [width of lens / 2] ))
    t = linspace(-tmax,tmax,500)
    xl = xL - Rc*cos(t) # centre of curvature for left side of lens -- radius of lens curvature * cos (t)
    yl = Rc*sin(t)
    xr = xR + Rc*cos(t)
    yr = Rc*sin(t)
    plot([xl,fliplr(xr),xl(1)],[yl,fliplr(yr),yl(1)],'b')

    #Plot lens foci
    plot(f.lens,0,'b*','markersize',fsize)
    plot(-f.lens,0,'b*','markersize',fsize)

    #Define a particular location on image to draw rays from
    d.a = x0
    d.b = 0.5*w*aspect_ratio + y0

    #Determine image location due to action of lens
    [d.xx,d.yy] = lens( d.x,d.y,f.lens )
    [d.aa,d.bb] = lens( d.a,d.b,f.lens )

    #Plot transformed image
    d.Imag = surf(d.xx,d.yy,-ones(height,width),orig_img)
    shading interp

    #Overlay rays from (d.a,d.b) to (d.aa,d.bb) via mirror
    d.ray1 = plot( [d.a,0,d.aa],[d.b,d.b,d.bb],'r-' )
    d.ray2 = plot( [d.a,d.aa],[d.b,d.bb],'r-' )
    d.ab = plot( d.a,d.b,'r*' )
    d.aabb = plot( d.aa,d.bb,'g*' )


#Function which executes when a key is pressed
def keypress(fig,event):
    global d
    if strcmp(get(fig,'currentkey'),'uparrow')==1:
        y.max = y.max + d.delta
        #Check that object is not beyond top of lens
        if y.max > h.lens:
            y.max = y.max - d.delta
        else:
            d.y = d.y + d.delta
            d.b = d.b + d.delta
            y.min = y.min + d.delta
            update()
    elif strcmp(get(fig,'currentkey'),'downarrow')==1:
        y.min = y.min - d.delta
        #Check that object is not below bottom of lens
        if y.min < -h.lens:
            y.min = y.min + d.delta
        else:
            d.y = d.y - d.delta
            d.b = d.b - d.delta
            y.max = y.max - d.delta
            update()
    elif strcmp(get(fig,'currentkey'),'leftarrow')==1:
        x.min = x.min - d.delta
        #Check that object is not within focal length
        if x.min < f.lens:
            x.min = x.min + d.delta
        else:
            d.x = d.x - d.delta
            d.a = d.a - d.delta
            update()
    elif strcmp(get(fig,'currentkey'),'rightarrow')==1:
        d.x = d.x + d.delta
        d.a = d.a + d.delta
        x.min = x.min + d.delta
        update()
    elif strcmp(get(fig,'currentkey'),'p')==1:
        print( d.fig, 'converging_lens.png','-dpng','-r300' )
    elif strcmp(get(fig,'currentkey'),'r')==1:
        if d.overlay_rays==1:
            d.overlay_rays = 0
            set(d.ray1,'visible','off')
            set(d.ray2,'visible','off')
            set(d.ab,'visible','off')
            set(d.aabb,'visible','off')
        else:
            d.overlay_rays = 1
            set(d.ray1,'visible','on')
            set(d.ray2,'visible','on')
            set(d.ab,'visible','on')
            set(d.aabb,'visible','on')
        update()
##

#Update screen
def update():
    global d

    #Determine transformation in mirror
    [d.xx,d.yy] = lens( d.x,d.y,f.lens )
    [d.aa,d.bb] = lens( d.a,d.b,f.lens )

    #Update object and image
    set( d.Obj,'xdata',d.x,'ydata',d.y )
    set( d.Imag,'xdata',d.xx,'ydata',d.yy )
    set( d.ray1,'xdata',[d.a,0,d.aa],'ydata',[d.b,d.b,d.bb])
    set(d.ray2,'xdata',[d.a,d.aa],'ydata',[d.b,d.bb])
    set(d.ab,'xdata',d.a,'ydata',d.b)
    set(d.aabb,'xdata',d.aa,'ydata',d.bb)
    drawnow

    ##

    # Lens image position (assume this lens equation 1/x + 1/xx = 1/f)
    # and yy/y = (xx - f)/x
    function [xx,yy] = lens( x,y,f )
    xx = -(( -1./x + 1/f).^-1)
    yy = y.*xx ./x

#End of code